## Assessment 1: Deep Learning

1) Answer all questions.
2) This assessment is open-book. You are allowed to refer to any references including online materials, books, notes, codes, github links, etc.
3) Copy this notebook to your google drive (click **FILE** > **save a copy in Drive**)
4) Upload the answer notebook to your github. 
5) Submit the assessment by sharing the link to your answer notebook. 





**QUESTION 1** 

One day while wandering around a clothing store at KL East Mall, you stumbled upon a pretty girl who is choosing a dress for Hari Raya. It turns out that the girl is visually impaired and had a hard time distinguishing between an abaya and a kebaya. To help people with the similar situation, you then decided to develop an AI system to identify the type of clothes using a Convolutional Neural Networks (ConvNet). In order to train the network, you decide to use the Fashion MNIST dataset which is freely available on Pytorch.


a) Given the problem, what is the most appropriate loss function to use? Justify your answer. **[5 marks]**

ANSWER: **Cross-entropy. Cross-entropy is the default loss function to use for a multi-class classification problem and our problem is to classify the types of clothes.**

b) Create and train a ConvNet corresponding to the following CNN architecture (with a modification of the final layer to address the number of classes). Please include **[10 marks]**:

    1) The dataloader to load the train and test datasets.

    2) The model definition (either using sequential method OR pytorch class method).

    3) Define your training loop.

    4) Output the mean accuracy for the whole testing dataset.

    



![pic](https://raw.githubusercontent.com/CUTe-EmbeddedAI/images/main/images/LeNet.png)

In [1]:
import torch, torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import glob
import numpy
import random

from PIL import Image
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
from torchsummary import summary

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.autograd import Variable

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix


In [2]:
transform = transforms.Compose(
    [transforms.Resize(32),
     transforms.ToTensor(),
     transforms.Normalize((0.5), (0.5))])

batch_size = 16


trainset = torchvision.datasets.FashionMNIST(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.FashionMNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('trouser', 'dress', 'abaya', 'kebaya', 'coat', 'sandal', 'shirt', 'sneaker', 'bag', 't-shirt/top')

FashionMNIST_labels = ["trouser",      # index 0
                       "dress",        # index 1
                       "abaya",        # index 2 
                       "kebaya",       # index 3 
                       "coat",         # index 4
                       "sandal",       # index 5
                       "shirt",        # index 6 
                       "sneaker",      # index 7 
                       "bag",          # index 8 
                       "t-shirt/top"]  # index 9

train_data_size = len(trainloader.dataset)
test_data_size = len(testloader.dataset)

print(train_data_size)
print(test_data_size)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw

60000
10000


In [7]:
#1. DEFINE THE CNN 
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)    #arguments: n_channels, out_channels, kernel_size
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2= nn.Conv2d(6, 16, 5)      
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [8]:
model = CNN() # need to instantiate the network to be used in instance method

# 2. LOSS AND OPTIMIZER
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# 3. move the model to GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)

CNN(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=10, bias=True)
  (relu): ReLU()
)

In [11]:
import time # to calculate training time

def train_and_validate(model, loss_criterion, optimizer, epochs=25):
    '''
    Function to train and validate
    Parameters
        :param model: Model to train and validate
        :param loss_criterion: Loss Criterion to minimize
        :param optimizer: Optimizer for computing gradients
        :param epochs: Number of epochs (default=25)
  
    Returns
        model: Trained Model with best validation accuracy
        history: (dict object): Having training loss, accuracy and validation loss, accuracy
    '''
    
    start = time.time()
    history = []
    best_acc = 0.0

    for epoch in range(epochs):
        epoch_start = time.time()
        print("Epoch: {}/{}".format(epoch+1, epochs))
        
        # Set to training mode
        model.train()
        
        # Loss and Accuracy within the epoch
        train_loss = 0.0
        train_acc = 0.0
        
        valid_loss = 0.0
        valid_acc = 0.0
        
        for i, (inputs, labels) in enumerate(trainloader):

            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # Clean existing gradients
            optimizer.zero_grad()
            
            # Forward pass - compute outputs on input data using the model
            outputs = model(inputs)
            
            # Compute loss
            loss = loss_criterion(outputs, labels)
            
            # Backpropagate the gradients
            loss.backward()
            
            # Update the parameters
            optimizer.step()
            
            # Compute the total loss for the batch and add it to train_loss
            train_loss += loss.item() * inputs.size(0)
            
            # Compute the accuracy
            ret, predictions = torch.max(outputs.data, 1)
            correct_counts = predictions.eq(labels.data.view_as(predictions))
            
            # Convert correct_counts to float and then compute the mean
            acc = torch.mean(correct_counts.type(torch.FloatTensor))
            
            # Compute total accuracy in the whole batch and add to train_acc
            train_acc += acc.item() * inputs.size(0)
            
            #print("Batch number: {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}".format(i, loss.item(), acc.item()))

            
        # Validation - No gradient tracking needed
        with torch.no_grad():

            # Set to evaluation mode
            model.eval()

            # Validation loop
            for j, (inputs, labels) in enumerate(testloader):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Forward pass - compute outputs on input data using the model
                outputs = model(inputs)

                # Compute loss
                loss = loss_criterion(outputs, labels)

                # Compute the total loss for the batch and add it to valid_loss
                valid_loss += loss.item() * inputs.size(0)

                # Calculate validation accuracy
                ret, predictions = torch.max(outputs.data, 1)
                correct_counts = predictions.eq(labels.data.view_as(predictions))

                # Convert correct_counts to float and then compute the mean
                acc = torch.mean(correct_counts.type(torch.FloatTensor))

                # Compute total accuracy in the whole batch and add to valid_acc
                valid_acc += acc.item() * inputs.size(0)

                #print("Validation Batch number: {:03d}, Validation: Loss: {:.4f}, Accuracy: {:.4f}".format(j, loss.item(), acc.item()))
            
        # Find average training loss and training accuracy
        avg_train_loss = train_loss/train_data_size 
        avg_train_acc = train_acc/train_data_size

        # Find average training loss and training accuracy
        avg_test_loss = valid_loss/test_data_size 
        avg_test_acc = valid_acc/test_data_size

        history.append([avg_train_loss, avg_test_loss, avg_train_acc, avg_test_acc])
                
        epoch_end = time.time()
    
        print("Epoch : {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}%, \n\t\tValidation : Loss : {:.4f}, Accuracy: {:.4f}%, Time: {:.4f}s".format(epoch, avg_train_loss, avg_train_acc*100, avg_test_loss, avg_test_acc*100, epoch_end-epoch_start))
        
        # Save if the model has best accuracy till now
        torch.save(model, 'fashion'+str(epoch)+'.pt')
            
    return model, history

In [ ]:
# 4. Train the model for 10 epochs

num_epochs = 10
trained_model, history = train_and_validate(model, criterion, optimizer, num_epochs)

Epoch: 1/10
Epoch : 000, Training: Loss: 0.7185, Accuracy: 73.9500%, 
		Validation : Loss : 0.5042, Accuracy: 81.2500%, Time: 33.8957s
Epoch: 2/10
Epoch : 001, Training: Loss: 0.4568, Accuracy: 83.2950%, 
		Validation : Loss : 0.4385, Accuracy: 84.3600%, Time: 26.4506s
Epoch: 3/10
Epoch : 002, Training: Loss: 0.3995, Accuracy: 85.3200%, 
		Validation : Loss : 0.3846, Accuracy: 86.0000%, Time: 26.3389s
Epoch: 4/10
Epoch : 003, Training: Loss: 0.3615, Accuracy: 86.7250%, 
		Validation : Loss : 0.3689, Accuracy: 86.7600%, Time: 26.0707s
Epoch: 5/10
Epoch : 004, Training: Loss: 0.3354, Accuracy: 87.7167%, 
		Validation : Loss : 0.3515, Accuracy: 86.9600%, Time: 27.6029s
Epoch: 6/10
Epoch : 005, Training: Loss: 0.3158, Accuracy: 88.3867%, 
		Validation : Loss : 0.3427, Accuracy: 87.4500%, Time: 27.7706s
Epoch: 7/10
Epoch : 006, Training: Loss: 0.2988, Accuracy: 89.0517%, 
		Validation : Loss : 0.3364, Accuracy: 87.4000%, Time: 26.4814s
Epoch: 8/10
Epoch : 007, Training: Loss: 0.2872, Accura

In [ ]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
#         images, labels = data
        images, labels = data[0].to(device), data[1].to(device)
        # calculate outputs by running images through the network
#         images = images.view(images.size(0), -1)
        outputs = model(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
#         images = images.view(images.size(0), -1)
        outputs = model(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

Accuracy of the network on the 10000 test images: 88 %
Accuracy for class: trouser is 87.5 %
Accuracy for class: dress is 97.4 %
Accuracy for class: abaya is 83.3 %
Accuracy for class: kebaya is 88.1 %
Accuracy for class: coat  is 86.5 %
Accuracy for class: sandal is 96.5 %
Accuracy for class: shirt is 59.4 %
Accuracy for class: sneaker is 97.2 %
Accuracy for class: bag   is 97.6 %
Accuracy for class: t-shirt/top is 94.2 %


c) Replace your defined CNN in b) with a pre-trained model. Then, proceed with a transfer learning and finetune the model for the Fashion MNIST dataset. **[10 marks]**

In [ ]:
transform = transforms.Compose(
    [transforms.Resize(224),
     transforms.ToTensor(),
     transforms.Lambda(lambda x: x.repeat(3, 1, 1)
     if x.size(0) == 1 else x),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# # batch_size
batch_size = 16

# # datasets
trainset = torchvision.datasets.FashionMNIST('./data',
     download=True,
     train=True,
     transform=transform)
testset = torchvision.datasets.FashionMNIST('./data',
     download=True,
     train=False,
     transform=transform)

# # dataloaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                         shuffle=True, num_workers=2)

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

# # constant for classes
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
            'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')

NameError: ignored

In [ ]:
model_ft = models.mobilenet_v2(pretrained=True)
num_ftrs = model_ft.classifier[1].in_features
# Here the size of each output sample is set to 10.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.classifier[1] = nn.Linear(num_ftrs, 10)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


  0%|          | 0.00/13.6M [00:00<?, ?B/s]

In [ ]:
model_ft.classifier[1]

Linear(in_features=1280, out_features=10, bias=True)

In [ ]:
num_epochs = 10
trained_model, history = train_and_validate(model_ft, criterion, optimizer_ft, num_epochs)

Epoch: 1/10
Epoch : 000, Training: Loss: 0.3213, Accuracy: 88.7833%, 
		Validation : Loss : 0.2354, Accuracy: 91.4100%, Time: 266.5303s
Epoch: 2/10
Epoch : 001, Training: Loss: 0.1975, Accuracy: 92.9717%, 
		Validation : Loss : 0.1906, Accuracy: 93.1500%, Time: 267.1216s
Epoch: 3/10
Epoch : 002, Training: Loss: 0.1596, Accuracy: 94.2633%, 
		Validation : Loss : 0.1669, Accuracy: 94.0100%, Time: 267.6290s
Epoch: 4/10
Epoch : 003, Training: Loss: 0.1369, Accuracy: 95.0567%, 
		Validation : Loss : 0.1700, Accuracy: 93.7900%, Time: 266.7542s
Epoch: 5/10
Epoch : 004, Training: Loss: 0.1160, Accuracy: 95.8383%, 
		Validation : Loss : 0.1749, Accuracy: 93.9100%, Time: 270.1789s
Epoch: 6/10
Epoch : 005, Training: Loss: 0.1018, Accuracy: 96.3000%, 
		Validation : Loss : 0.1895, Accuracy: 93.3000%, Time: 267.8836s
Epoch: 7/10
Epoch : 006, Training: Loss: 0.0884, Accuracy: 96.7917%, 
		Validation : Loss : 0.1919, Accuracy: 93.9700%, Time: 267.5319s
Epoch: 8/10
Epoch : 007, Training: Loss: 0.0777,

d) Using model-centric methods, propose two (2) strategies that can be used to increase the accuracy of the model on the testing dataset. **[5 marks]**

Two model-centric techniques that I propose are:

**1)Batch Normalization** 
**2)Dropout**


<span style="color:blue">
    Two model-centric techniques that I propose are: </span>

e) Next, implement the two proposed model-centric techniques for the same problem as in the previous question. **[15 marks]**

In [ ]:
transform = transforms.Compose(
    [transforms.Resize(32),
     transforms.ToTensor(),
     transforms.Normalize((0.5), (0.5))])

batch_size = 16

trainset = torchvision.datasets.FashionMNIST(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.FashionMNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)


train_data_size = len(trainloader.dataset)
test_data_size = len(testloader.dataset)

print(train_data_size)
print(test_data_size)

60000
10000


In [ ]:
class CNN2(nn.Module):
    def __init__(self):
        super(CNN2, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)  #arguments: n_channels, out_channels, kernel_size
        self.pool = nn.MaxPool2d(2, 2)
        self.batchnorm = nn.BatchNorm2d(6)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.dropout = nn.Dropout(p=0.2)
        self.fc2 = nn.Linear(120, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.batchnorm(x)
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [ ]:
model2 = CNN2() # need to instantiate the network to be used in instance method

# 2. LOSS AND OPTIMIZER
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model2.parameters(), lr=0.001, momentum=0.9)

# 3. move the model to GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model2.to(device)

CNN2(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (batchnorm): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=120, out_features=10, bias=True)
  (relu): ReLU()
)

In [ ]:
# 4. Train the model for 10 epochs

num_epochs = 10
trained_model, history = train_and_validate(model2, criterion, optimizer, num_epochs)

Epoch: 1/10
Epoch : 000, Training: Loss: 0.6066, Accuracy: 77.7933%, 
		Validation : Loss : 0.4219, Accuracy: 84.7200%, Time: 26.8298s
Epoch: 2/10
Epoch : 001, Training: Loss: 0.3853, Accuracy: 85.9133%, 
		Validation : Loss : 0.3674, Accuracy: 86.8100%, Time: 26.0877s
Epoch: 3/10
Epoch : 002, Training: Loss: 0.3412, Accuracy: 87.4500%, 
		Validation : Loss : 0.3329, Accuracy: 87.6700%, Time: 26.1529s
Epoch: 4/10
Epoch : 003, Training: Loss: 0.3127, Accuracy: 88.4733%, 
		Validation : Loss : 0.3177, Accuracy: 88.4600%, Time: 26.8592s
Epoch: 5/10
Epoch : 004, Training: Loss: 0.2923, Accuracy: 89.2967%, 
		Validation : Loss : 0.3042, Accuracy: 88.8100%, Time: 27.1368s
Epoch: 6/10
Epoch : 005, Training: Loss: 0.2797, Accuracy: 89.5167%, 
		Validation : Loss : 0.2957, Accuracy: 89.0300%, Time: 26.0532s
Epoch: 7/10
Epoch : 006, Training: Loss: 0.2652, Accuracy: 89.9833%, 
		Validation : Loss : 0.2841, Accuracy: 89.5700%, Time: 25.9416s
Epoch: 8/10
Epoch : 007, Training: Loss: 0.2564, Accura

f) Do you see any accuracy improvement? Whether it is a "yes" or "no", discuss the possible reasons contributing to the accuracy improvement/ unimprovement. **[5 marks]**

**No,** there is no improvement in the accuracy between the first CNN model and the CNN model with batch normalization and dropout. Since the batch size is small, batch normalization may cause inaccurate estimation of batch statistics which increases the model error.

<span style="color:blue">
    Your answer here </span>

g) In real applications, data-centric strategies are essential to train robust deep learning models. Give two (2) examples of such strategies and discuss how the strategies helps improving the model accuracy. **[5 marks]**

1) **Random invert** and
2) **Random resized crop**

<span style="color:blue">
    Your answer here </span>

h) Next, implement the two proposed data-centric techniques for the same problem as in the previous question. **[10 marks]**

In [3]:
transform = transforms.Compose(
    [transforms.RandomInvert(),
     transforms.RandomResizedCrop(size=(32, 32)),
     transforms.ToTensor(),
     transforms.Normalize((0.5), (0.5))])

batch_size = 16

trainset = torchvision.datasets.FashionMNIST(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.FashionMNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)


train_data_size = len(trainloader.dataset)
test_data_size = len(testloader.dataset)

print(train_data_size)
print(test_data_size)

60000
10000


In [13]:
class CNN2(nn.Module):
    def __init__(self):
        super(CNN2, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)  #arguments: n_channels, out_channels, kernel_size
        self.pool = nn.MaxPool2d(2, 2)
        self.batchnorm = nn.BatchNorm2d(6)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.dropout = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(120, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.batchnorm(x)
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [9]:
model3 = CNN() # need to instantiate the network to be used in instance method

# 2. LOSS AND OPTIMIZER
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model3.parameters(), lr=0.001, momentum=0.9)

# 3. move the model to GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model3.to(device)

CNN(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=10, bias=True)
  (relu): ReLU()
)

In [15]:
# 4. Train the model for 10 epochs

num_epochs = 10
trained_model, history = train_and_validate(model3, criterion, optimizer, num_epochs)

Epoch: 1/10
Epoch : 000, Training: Loss: 0.9763, Accuracy: 63.9100%, 
		Validation : Loss : 0.9662, Accuracy: 63.9800%, Time: 34.5262s
Epoch: 2/10
Epoch : 001, Training: Loss: 0.9451, Accuracy: 65.3400%, 
		Validation : Loss : 0.9277, Accuracy: 65.8200%, Time: 35.1347s
Epoch: 3/10
Epoch : 002, Training: Loss: 0.9210, Accuracy: 66.2700%, 
		Validation : Loss : 0.9257, Accuracy: 66.2700%, Time: 35.6507s
Epoch: 4/10
Epoch : 003, Training: Loss: 0.8986, Accuracy: 66.9183%, 
		Validation : Loss : 0.9010, Accuracy: 66.7100%, Time: 34.5709s
Epoch: 5/10
Epoch : 004, Training: Loss: 0.8825, Accuracy: 67.3917%, 
		Validation : Loss : 0.8776, Accuracy: 67.5700%, Time: 34.6383s
Epoch: 6/10
Epoch : 005, Training: Loss: 0.8668, Accuracy: 68.0000%, 
		Validation : Loss : 0.8558, Accuracy: 68.9200%, Time: 35.5697s
Epoch: 7/10
Epoch : 006, Training: Loss: 0.8544, Accuracy: 68.4117%, 
		Validation : Loss : 0.8505, Accuracy: 69.1300%, Time: 34.7785s
Epoch: 8/10
Epoch : 007, Training: Loss: 0.8417, Accura

In [14]:
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
            'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')

In [16]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
#         images, labels = data
        images, labels = data[0].to(device), data[1].to(device)
        # calculate outputs by running images through the network
#         images = images.view(images.size(0), -1)
        outputs = model3(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
#         images = images.view(images.size(0), -1)
        outputs = model3(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

Accuracy of the network on the 10000 test images: 67 %
Accuracy for class: T-shirt/top is 52.2 %
Accuracy for class: Trouser is 92.4 %
Accuracy for class: Pullover is 52.0 %
Accuracy for class: Dress is 71.8 %
Accuracy for class: Coat  is 42.3 %
Accuracy for class: Sandal is 84.5 %
Accuracy for class: Shirt is 51.3 %
Accuracy for class: Sneaker is 75.8 %
Accuracy for class: Bag   is 83.3 %
Accuracy for class: Ankle Boot is 79.3 %


**QUESTION 2** **[35 marks]**

Firstly, watch this video:

https://drive.google.com/file/d/1bsypahR7I3f_R3DXkfw_tf0BrbCHxE_O/view?usp=sharing

This video shows an example of masked face recognition where the deep learning model is able to detect and classify your face even when wearing a face mask. Using the end-to-end object detection pipeline that you have learned, develop your own masked face recognition such that the model should recognize your face even on face mask while recognize other persons as "others".

Deliverables for this question are:

- the model file. Change the name to <your_name>.pt file (e.g. hasan.pt).
- a short video (~10 secs) containing your face and your friends faces (for inference).

In [ ]:
##https://github.com/zatulnatasha/Deep-Learning-Exam